<h1>Internship Recommendation: MissionED</h1>

<h2>Step-1 : Category Classifier</h2>

***Importing the libraries***

In [59]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

***Importing the dataset***

In [60]:
dataset = pd.read_excel('Internship Naming.xlsx', sheet_name='Duration')
dataset = dataset.dropna(axis=1, how='all')

In [61]:
dataset.head(3)

,Description,Code
0,"Education Requirements:Stipend: 10,000/month\n...",1
1,About Eternal Robotics\n\nWebsite\n\nActivity ...,1
2,\nActivity on Internshala\n\nHiring since Augu...,1


**Cleaning the texts**

In [62]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer 
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
tokenizer = RegexpTokenizer(r'\w+')
en_stopwords = set(stopwords.words('english'))
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ashis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
def getCleanedText(text):
  text = text.lower()
  #tokenizer
  tokens = tokenizer.tokenize(text)
  new_tokens = [token for token in tokens if token not in en_stopwords]
  stemmed_tokens = [ps.stem(tokens) for tokens in new_tokens]
  clean_text = ' '.join(stemmed_tokens)
  removed_numbers = "".join([i for i in clean_text if not i.isdigit()])
  return removed_numbers

***Splitting the dataset***

In [64]:
X = dataset['Description'].apply(getCleanedText)
y = dataset['Code']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

***Cleaning the x_train and x_test variables***

In [65]:
X_clean = [getCleanedText(i) for i in X_train]
X_clean_test = [getCleanedText(i) for i in X_test]
len(X_clean)

37

***Training the Support Vector model on the Training set***

In [66]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range = (1,2))
x_vec = cv.fit_transform(X_clean).toarray()
x_vec_test = cv.transform(X_clean_test).toarray()

In [67]:
#import the naive bayes model
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(x_vec, y_train)

MultinomialNB()

In [68]:
classifier.score(x_vec_test, y_test)

1.0

In [69]:
classifier.score(x_vec_test, y_test)

1.0

In [70]:
import joblib
joblib.dump(classifier, 'InternshipRecommender.pkl')

['InternshipRecommender.pkl']

In [76]:
#load the model from disk
loaded_model = joblib.load('InternshipRecommender.pkl')
result = loaded_model.score(x_vec_test, y_test)
details = "About the internship"
text = getCleanedText(details)
x_vec = cv.fit_transform([text]).toarray()
prediction = loaded_model.predict(x_vec)
prediction[0]

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3557 is different from 1)

In [72]:
import json
columns = {
    'data_columns' : [col.lower() for col in dataset.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))

In [73]:
def predictCategory(df):
  # iteraion through df data frame
  for i in range(len(df)):
    text = getCleanedText(df["Internship"][i])
    x_vec = cv.fit_transform([text]).toarray()
    df['Category'][i]= classifier.predict(x_vec)
  return df

In [74]:
df = pd.read_excel('Internship Naming.xlsx', sheet_name='Work From Home')
df = df.dropna(axis=1, how='all')
df['Category']=0

In [75]:
predictCategory(df)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3557 is different from 134)

In [ ]:
df["Duration"] = 0
df['Stipend'] =0
df['workfromhome'] =0
df['certificate'] = 0
df['flexible'] =0
df['LOR'] = 0
#drop code from dataframe
df

,Internship,Category,Duration,Stipend,workfromhome,certificate,flexible,LOR
0,About the work from home job/internship\nSelec...,1,0,0,0,0,0,0
1,About the work from home job/internship\nSelec...,0,0,0,0,0,0,0
2,About part time job/internship\nSelected inter...,1,0,0,0,0,0,0
3,About the work from home job/internship\nSelec...,1,0,0,0,0,0,0
4,About the work from home job/internship\nSelec...,1,0,0,0,0,0,0
5,About the work from home job/internship\nSelec...,1,0,0,0,0,0,0
6,About the work from home job/internship\nSelec...,1,0,0,0,0,0,0
7,About the work from home job/internship\nSelec...,1,0,0,0,0,0,0
8,About the internship\nSelected intern's day-to...,1,0,0,0,0,0,0
9,About the work from home job/internship\nSelec...,1,0,0,0,0,0,0


In [ ]:
def searchmonth():
    for i in range(len(df)):
        if df['Internship'][i].find('months') != -1:
            before_keyword, keyword, after_keyword = df['Internship'][i].partition('months')
            try:
                df['Duration'][i] = int(before_keyword.split()[-1])
            except:
                df['Duration'][i] = 0
        else:
            df['Duration'][i] = 0
searchmonth()

<ipython-input-20-ac2011e5474c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Duration'][i] = int(before_keyword.split()[-1])
<ipython-input-20-ac2011e5474c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Duration'][i] = 0
<ipython-input-20-ac2011e5474c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Duration'][i] = 0


In [ ]:
import re
def searchstipend():
    for i in range(len(df)):
        if 'stipend' in df['Internship'][i].lower():
            df['Internship'][i].lower().replace(',','')
            df['Internship'][i].replace('/',' ')
            df['Internship'][i].replace('-',' ')
            if 'unpaid' in df['Internship'][i]:
                df['Stipend'][i] = 0
            else:
                #selecting nearest integer after stipend word
                df['Stipend'][i] = 1
        else:
            df['Stipend'][i] = 0
searchstipend()

<ipython-input-21-4872ae3b4fa6>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Stipend'][i] = 0
<ipython-input-21-4872ae3b4fa6>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Stipend'][i] = 1


In [ ]:
remote  = 'remote'
workfromhome = 'work from home'
def searchremote():
    for i in range(len(df)):
        if (( remote in df['Internship'][i].lower()) | ('work from home' in df['Internship'][i].lower())):
            df['workfromhome'][i] = 1
        else:
            df['workfromhome'][i] = 0
    df['workfromhome']
searchremote()

<ipython-input-22-b1e6c674a41f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['workfromhome'][i] = 1
<ipython-input-22-b1e6c674a41f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['workfromhome'][i] = 0


In [ ]:
def searchcertificate():
    for i in range(len(df)):
        if 'certificate' in df['Internship'][i].lower():
            df['certificate'][i] = 1
        else:
            df['certificate'][i] = 0
searchcertificate()

<ipython-input-23-2844aa1d8911>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['certificate'][i] = 1
<ipython-input-23-2844aa1d8911>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['certificate'][i] = 0


In [ ]:
def searchLOR():
    for i in range(len(df)):
        if 'lor' in df['Internship'][i].lower() or 'letter of recommendation' in df['Internship'][i].lower():
            df['LOR'][i] = 1
        else:
            df['LOR'][i] = 0
searchLOR()

<ipython-input-24-19285ca9d222>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LOR'][i] = 1
<ipython-input-24-19285ca9d222>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LOR'][i] = 0


In [ ]:
def searchFlexible():
    for i in range(len(df)):
        if 'flexible' in df['Internship'][i].lower():
            df['flexible'][i] = 1
        else:
            df['flexible'][i] = 0
searchFlexible()

<ipython-input-25-30410dbbad4b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['flexible'][i] = 0
<ipython-input-25-30410dbbad4b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['flexible'][i] = 1


In [ ]:
# join duration, stipend, workfromhome, certificate, LOR, flexible in code column
df['Code'] = df['Category'].astype(str) + '' +df['Duration'].astype(str) + '' + df['Stipend'].astype(str) + '' + df['workfromhome'].astype(str) + '' + df['certificate'].astype(str) + '' + df['LOR'].astype(str) + '' + df['flexible'].astype(str)

In [ ]:
#write code column in descending order
dfsorted = df.sort_values(by=['Code'], ascending=False)
# drop index from index in dfsorted
dfsorted = dfsorted.reset_index(drop=True)
dfsorted

,Internship,Category,Duration,Stipend,workfromhome,certificate,flexible,LOR,Code
0,About the work from home job/internship\nSelec...,1,6,0,1,1,1,1,1601111
1,About the work from home job/internship\nSelec...,1,6,0,1,1,1,1,1601111
2,About the internship\nSelected intern's day-to...,1,6,0,0,1,1,1,1600111
3,Job Requirements\n\nJOB TITLE - Machine Learni...,1,4,0,0,0,0,0,1400000
4,About the work from home job/internship\nSelec...,1,3,1,1,1,1,1,1311111
5,About the work from home job/internship\nSelec...,1,3,0,1,1,1,1,1301111
6,About the work from home job/internship\nSelec...,1,2,0,1,1,1,1,1201111
7,About the work from home job/internship\nSelec...,1,2,0,1,1,0,1,1201110
8,About the work from home job/internship\nSelec...,1,2,0,1,1,0,1,1201110
9,About the work from home job/internship\nSelec...,1,2,0,1,1,0,1,1201110


In [ ]:
# drop columsn for duration, stipend, workfromhome, certificate, LOR, flexible
dfsorted = dfsorted.drop(['Duration','Stipend','workfromhome','certificate','LOR','flexible','Category'], axis=1)

In [ ]:
dfsorted

,Internship,Code
0,About the work from home job/internship\nSelec...,1601111
1,About the work from home job/internship\nSelec...,1601111
2,About the internship\nSelected intern's day-to...,1600111
3,Job Requirements\n\nJOB TITLE - Machine Learni...,1400000
4,About the work from home job/internship\nSelec...,1311111
5,About the work from home job/internship\nSelec...,1301111
6,About the work from home job/internship\nSelec...,1201111
7,About the work from home job/internship\nSelec...,1201110
8,About the work from home job/internship\nSelec...,1201110
9,About the work from home job/internship\nSelec...,1201110
